# TensorFlow 2.0

In [1]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt

%load_ext tensorboard
%matplotlib inline

## Hyperparameter Tunning

In [2]:
num_epochs = 100
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

# Data Preprocess

In [5]:
train_paths = glob('../dataset/cifar/train/*.png')[:1000]
test_paths = glob('../dataset/cifar/test/*.png')[:100]

In [6]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [7]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [8]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [9]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [10]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [11]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [13]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

## Training

http://localhost:6006

In [14]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

history = model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs
)

C:\Users\jumin\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
31/31 [==============================] - 8s 175ms/step - loss: 2.3145 - accuracy: 0.1043 - val_loss: 2.2907 - val_accuracy: 0.0729
Epoch 2/100
31/31 [==============================] - 5s 166ms/step - loss: 2.2940 - accuracy: 0.1322 - val_loss: 2.2850 - val_accuracy: 0.0625
Epoch 3/100
31/31 [==============================] - 5s 155ms/step - loss: 2.2272 - accuracy: 0.1581 - val_loss: 2.0969 - val_accuracy: 0.2917
Epoch 4/100
31/31 [==============================] - 5s 162ms/step - loss: 2.1672 - accuracy: 0.1932 - val_loss: 2.1068 - val_accuracy: 0.2188
Epoch 5/100
31/31 [==============================] - 5s 147ms/step - loss: 2.0907 - accuracy: 0.2345 - val_loss: 2.1008 - val_accuracy: 0.2604
Epoch 6/100
31/31 [==============================] - 5s 157ms/step - loss: 2.0964 - accuracy: 0.2097 - val_loss: 2.0857 - val_accuracy: 0.1562
Epoch 7/100
31/31 [==============================] - 6s 182ms/step - loss: 1.9720 - accuracy: 0.2617 - val_loss: 2.0867 - val_accuracy: 0.2188

Epoch 58/100
31/31 [==============================] - 4s 137ms/step - loss: 0.7848 - accuracy: 0.7180 - val_loss: 1.6795 - val_accuracy: 0.5417
Epoch 59/100
31/31 [==============================] - 4s 134ms/step - loss: 0.8190 - accuracy: 0.7066 - val_loss: 1.6953 - val_accuracy: 0.5417
Epoch 60/100
31/31 [==============================] - 4s 138ms/step - loss: 0.7919 - accuracy: 0.7288 - val_loss: 1.6707 - val_accuracy: 0.5104
Epoch 61/100
31/31 [==============================] - 4s 133ms/step - loss: 0.7936 - accuracy: 0.7150 - val_loss: 1.7304 - val_accuracy: 0.5521
Epoch 62/100
31/31 [==============================] - 4s 139ms/step - loss: 0.8218 - accuracy: 0.7198 - val_loss: 1.8365 - val_accuracy: 0.5104
Epoch 63/100
31/31 [==============================] - 4s 137ms/step - loss: 0.7300 - accuracy: 0.7438 - val_loss: 1.8336 - val_accuracy: 0.5208
Epoch 64/100
31/31 [==============================] - 4s 137ms/step - loss: 0.6871 - accuracy: 0.7645 - val_loss: 1.8457 - val_accuracy:

# Saving Model
- 전부 저장

In [16]:
save_path = 'my_model.h5'

In [18]:
model.save(save_path, include_optimizer=True)

- 모델 불러오기

In [20]:
load_model = tf.keras.models.load_model('my_model.h5')

# Saving Model 
- weight만 저장함
- 후에, transfer learning 할 때 유용

In [22]:
model.save_weights("model_weights.h5")

- model archtecture만 저장 및 불러오기

In [21]:
with open("model_architecture.json", "w") as f:
    f.write(model.to_json())

In [27]:
from tensorflow.keras.models import model_from_json

In [28]:
with open("model_architecture.json", 'r') as f :
    model = model_from_json(f.read())

- weight 가져오기: h5 파일 또는 checkpoints

In [30]:
model.load_weights("model_weights.h5")

In [ ]:
model.load_weights('checkpoint')  #checkpoint : checkpoint가 담긴 파일

# model.h5 들여다보기

In [31]:
import h5py

In [33]:
model_file = h5py.File("my_model.h5", 'r+')

In [35]:
model_file['model_weights'].keys()

<KeysViewHDF5 ['activation', 'activation_1', 'activation_2', 'activation_3', 'activation_4', 'activation_5', 'conv2d', 'conv2d_1', 'conv2d_2', 'conv2d_3', 'dense', 'dense_1', 'dropout', 'dropout_1', 'dropout_2', 'flatten', 'input_1', 'max_pooling2d', 'max_pooling2d_1']>

In [36]:
model_file['model_weights']['conv2d']['conv2d'].keys()

<KeysViewHDF5 ['bias:0', 'kernel:0']>

In [40]:
weight = model_file['model_weights']['conv2d']['conv2d']['kernel:0']

In [41]:
weight

<HDF5 dataset "kernel:0": shape (3, 3, 3, 32), type "<f4">